# STACKING, BAGGING AND BOOSTING

In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv("C:/Users/sachin/OneDrive/Desktop/datasets/tested.csv")
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,S


# cleaning and preprosessing data


In [3]:
df.shape

(418, 9)

In [4]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Embarked        object
dtype: object

In [5]:
del df['Name']

In [6]:
del df['Sex']

In [7]:
del df['Embarked']

In [8]:
df

,PassengerId,Survived,Pclass,Age,SibSp,Parch
0,892,0,3,34.5,0,0
1,893,1,3,47.0,1,0
2,894,0,2,62.0,0,0
3,895,0,3,27.0,0,0
4,896,1,3,22.0,1,1
...,...,...,...,...,...,...
413,1305,0,3,NaN,0,0
414,1306,1,1,39.0,0,0
415,1307,0,3,38.5,0,0
416,1308,0,3,NaN,0,0


In [9]:
df.isnull().sum()

PassengerId     0
Survived        0
Pclass          0
Age            86
SibSp           0
Parch           0
dtype: int64

In [10]:
df.dropna(inplace = True)

In [11]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
dtype: int64

# Ques 1 Implement Random Forest using stacking technique. 

In [12]:
x = df.drop(["Survived"],axis = 1)
y = df["Survived"]


In [13]:
from sklearn.model_selection import train_test_split as tts

train_x, test_x,train_y,test_y = tts(x,y,random_state = 9, stratify = y)


In [14]:
#base model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold

In [15]:
# model training and predictions
#k-fold validation
def kfold_pred(model, n_splits, train_x, train_y, test_x):
    train_pred = np.empty((0,0) , int)
    skfold = KFold(n_splits)
 
    
    for i,j in skfold.split(train_x, train_y):
        x_train, x_test = train_x.iloc[i], train_x.iloc[j]
        y_train, y_test = train_y.iloc[i], train_y.iloc[j]
 
   
        model.fit( X = x_train, y = y_train)
 
    
        tmp = model.predict(x_test)
 
    
        train_pred = np.append(train_pred, tmp)
 
    
    model.fit(train_x, train_y)
    test_pred = model.predict(test_x)
 
    return train_pred, test_pred

from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
M1_train, M1_test = kfold_pred( LR, 5, train_x, train_y, 
test_x) 
    
    

In [16]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
M2_train, M2_test = kfold_pred(DT, 10, train_x, train_y, 
test_x) 
#dt_train_pred.shape, dt_test_pred.shape, data_x.shape, data_y.shape
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X=train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)
train_x = pd.DataFrame(train_x)
test_x = pd.DataFrame(test_x)
# Base K-Neighbors Model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
M3_train, M3_test= kfold_pred( knn, 10, train_x, train_y, 
test_x) 
#knn_train_pred.shape, knn_test_pred.shape, data_x.shape, data_y.shape
# Arranging Train KFold Predictions
train_prediction = {
 'LR': M1_train,
 'KNN': M3_train,
 'DT': M2_train
 }
train_predictions = pd.DataFrame(train_prediction)
train_predictions.head()


,LR,KNN,DT
0,0,1,0
1,0,1,0
2,0,0,0
3,0,0,0
4,0,0,0


In [17]:
# Arranging test KFold Predictions
test_prediction = {
 'LR': M1_test,
 'KNN': M3_test,
 'DT': M2_test
 }
test_predictions = pd.DataFrame(test_prediction)
test_predictions.head()

,LR,KNN,DT
0,1,1,0
1,0,1,1
2,0,1,1
3,0,0,0
4,0,0,0


In [18]:
model = KNeighborsClassifier()
model.fit(train_predictions, train_y)
final = model.predict(test_predictions)
model.score(test_predictions,test_y)

0.6385542168674698

# ques 3. Implement Random Forest using bagging/bootstrap aggregating technique. 

In [19]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state = 96,n_estimators = 100,max_depth = 10)
clf.fit(train_x,train_y)

RandomForestClassifier(max_depth=10, random_state=96)

In [20]:
print("Training score:",round(clf.score(train_x,train_y),4))

Training score: 0.9759


In [21]:
y_pred = clf.predict(test_x)

In [22]:
#model evaluation
from sklearn.metrics import accuracy_score,classification_report
print("model accuraccy score",round(accuracy_score(test_y,y_pred),4))

model accuraccy score 0.6024


In [23]:
print(classification_report(test_y,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.80      0.71        51
           1       0.47      0.28      0.35        32

    accuracy                           0.60        83
   macro avg       0.56      0.54      0.53        83
weighted avg       0.58      0.60      0.57        83



# ques 4.Implement Random Forest using gradient boosting technique. 

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [25]:
gbc = GradientBoostingClassifier(n_estimators=100,learning_rate=0.08,random_state=96,max_features=5)
gbc.fit(train_x,train_y)

GradientBoostingClassifier(learning_rate=0.08, max_features=5, random_state=96)

In [26]:
print("training score:", round(gbc.score(train_x,train_y),4))

training score: 0.8554


In [27]:
print(classification_report(test_y,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.80      0.71        51
           1       0.47      0.28      0.35        32

    accuracy                           0.60        83
   macro avg       0.56      0.54      0.53        83
weighted avg       0.58      0.60      0.57        83



# ques 6.Implement Random Forest using AdaBoost technique. 

In [28]:
from sklearn.ensemble import AdaBoostClassifier
adb = AdaBoostClassifier()
adb.fit(train_x,train_y)

AdaBoostClassifier()

In [29]:
print("training score:", round(adb.score(train_x,train_y),4))

training score: 0.7831


In [30]:
print(classification_report(test_y,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.80      0.71        51
           1       0.47      0.28      0.35        32

    accuracy                           0.60        83
   macro avg       0.56      0.54      0.53        83
weighted avg       0.58      0.60      0.57        83



In [31]:
print("accuracy score:",adb.score(test_x,test_y))

accuracy score: 0.6144578313253012
